In [13]:
import argparse
import sys
sys.path.append('/datacommons/carlsonlab/yl407/packages')
sys.path.insert(0, '../../')
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor
from tqdm import tqdm
import torch.nn.functional as F
import utils.config as config
from utils.helper import *
from models.model_mnist_mnistm import *
from data.mnist_mnistm_data import *
import pandas as pd
import random
import time
import warnings
import argparse
import shutil
import os.path as osp
import os
from utils.helperOH import *
import data.officehome as datasets
import models.model_resnet as models
import timm
from torch.utils.data import DataLoader
from models.model_office_home import *

#seed_torch(123) # 0 #123
device = 'cpu'

In [22]:
train_resizing = 'default'
val_resizing = 'default'

train_transform = get_train_transform(train_resizing, random_horizontal_flip=False,
                                      random_color_jitter=False, resize_size=224,
                                      norm_mean=(0.485, 0.456, 0.406), norm_std=(0.229, 0.224, 0.225))
val_transform = get_val_transform(val_resizing, resize_size=False,
                                  norm_mean=(0.485, 0.456, 0.406), norm_std=(0.229, 0.224, 0.225))
train_source_dataset, train_target_dataset, val_dataset, test_dataset, num_classes, class_names = \
get_dataset(dataset_name='OfficeHome', root='/datacommons/carlsonlab/yl407/office_home_imbalance',\
            source={'Ar': 'image_list/Art.txt'}, target={'Cl': 'image_list/Clipart.txt'}, \
            train_source_transform=train_transform, val_transform=val_transform)

In [23]:
len(train_source_dataset)

2427

In [ ]:
len(train_source_dataset)

In [ ]:
import torch

# create an empty tensor with size (0, 3)
empty_tensor = torch.empty(0, 3)

# create two tensors to concatenate
tensor1 = torch.tensor([[1, 2, 3]])
tensor2 = torch.tensor([[4, 5, 6]])

# concatenate the tensors along the first dimension (rows)
concatenated_tensor = torch.cat((empty_tensor, tensor1, tensor2), dim=0)

print(concatenated_tensor)

In [ ]:
num_classes = 65
class_indices = [[] for _ in range(num_classes)]
for i in range(len(train_source_dataset)):
    label = train_source_dataset[i][1]
    class_indices[label].append(i)

In [ ]:
class_indices

In [ ]:
# Create a custom batch sampler that groups the data by class
batch_size = 32
num_batches_per_epoch = len(my_dataset) // batch_size
batch_indices = []
for i in range(num_batches_per_epoch):
    batch_indices.append([])
    for j in range(num_classes):
        indices = class_indices[j]
        num_samples = len(indices)
        idx = i * num_classes + j
        batch_indices[-1].append(indices[idx % num_samples])